In [1]:
import pandas as pd
import numpy as np

In [2]:
# Dataset de ventas
ventas = pd.DataFrame({
    'id_venta': range(1, 11),
    'id_cliente': np.random.choice([1, 2, 3, 4, 5], 10),
    'id_producto': np.random.choice([101, 102, 103, 104], 10),
    'cantidad': np.random.randint(1, 5, 10),
    'fecha': pd.date_range('2024-01-01', periods=10, freq='D')
})

# Dataset de clientes
clientes = pd.DataFrame({
    'id_cliente': [1, 2, 3, 4, 5],
    'nombre': ['Ana', 'Carlos', 'María', 'Juan', 'Luis'],
    'ciudad': ['Madrid', 'Barcelona', 'Madrid', 'Valencia', 'Sevilla']
})

# Dataset de productos
productos = pd.DataFrame({
    'id_producto': [101, 102, 103, 104],
    'nombre': ['Laptop', 'Mouse', 'Teclado', 'Monitor'],
    'precio': [1200, 25, 80, 300],
    'categoria': ['Electrónica', 'Accesorios', 'Accesorios', 'Electrónica']
})

In [3]:
print("Datasets creados:")
print(f"Ventas: {ventas.shape}")
print(f"Clientes: {clientes.shape}")
print(f"Productos: {productos.shape}")

Datasets creados:
Ventas: (10, 5)
Clientes: (5, 3)
Productos: (4, 4)


In [4]:
# Ventas del mes actual con query
ventas_recientes = ventas.query('fecha >= "2024-01-05"')

print("Ventas recientes:")
print(ventas_recientes)
print("\nCantidad de ventas recientes:", len(ventas_recientes))

Ventas recientes:
   id_venta  id_cliente  id_producto  cantidad      fecha
4         5           3          104         4 2024-01-05
5         6           5          102         3 2024-01-06
6         7           2          102         3 2024-01-07
7         8           5          102         1 2024-01-08
8         9           4          104         4 2024-01-09
9        10           5          102         1 2024-01-10

Cantidad de ventas recientes: 6


In [5]:
# Productos caros usando variable externa
precio_limite = 100

productos_caros = productos.query('precio >= @precio_limite')

print("Productos caros:")
print(productos_caros)

Productos caros:
   id_producto   nombre  precio    categoria
0          101   Laptop    1200  Electrónica
3          104  Monitor     300  Electrónica


In [6]:
# Ventas por producto
ventas_por_producto = ventas.groupby('id_producto')['cantidad'].sum()

print("Ventas totales por producto:")
print(ventas_por_producto)

Ventas totales por producto:
id_producto
102     9
103     5
104    10
Name: cantidad, dtype: int32


In [7]:
# Estadísticas por cliente
stats_por_cliente = ventas.groupby('id_cliente').agg({
    'cantidad': ['sum', 'mean'],
    'id_venta': 'count'
})

print("Estadísticas por cliente:")
print(stats_por_cliente)

Estadísticas por cliente:
           cantidad      id_venta
                sum mean    count
id_cliente                       
1                 5  2.5        2
2                 5  2.5        2
3                 4  4.0        1
4                 4  4.0        1
5                 6  1.5        4


In [8]:
# Unir ventas con productos
ventas_productos = pd.merge(ventas, productos, on='id_producto')

print("Ventas + Productos (primeras filas):")
ventas_productos.head()

Ventas + Productos (primeras filas):


,id_venta,id_cliente,id_producto,cantidad,fecha,nombre,precio,categoria
0,1,2,103,2,2024-01-01,Teclado,80,Accesorios
1,2,1,103,3,2024-01-02,Teclado,80,Accesorios
2,3,5,102,1,2024-01-03,Mouse,25,Accesorios
3,4,1,104,2,2024-01-04,Monitor,300,Electrónica
4,5,3,104,4,2024-01-05,Monitor,300,Electrónica


In [9]:
# Calcular totales
ventas_productos['total'] = ventas_productos['cantidad'] * ventas_productos['precio']

print("Ventas con total calculado:")
ventas_productos[['id_venta', 'nombre', 'cantidad', 'precio', 'total']].head()

Ventas con total calculado:


,id_venta,nombre,cantidad,precio,total
0,1,Teclado,2,80,160
1,2,Teclado,3,80,240
2,3,Mouse,1,25,25
3,4,Monitor,2,300,600
4,5,Monitor,4,300,1200


In [10]:
# Unir con clientes
analisis_completo = pd.merge(
    ventas_productos,
    clientes,
    on='id_cliente'
)

print("Análisis completo (primeras filas):")
analisis_completo.head()

Análisis completo (primeras filas):


,id_venta,id_cliente,id_producto,cantidad,fecha,nombre_x,precio,categoria,total,nombre_y,ciudad
0,1,2,103,2,2024-01-01,Teclado,80,Accesorios,160,Carlos,Barcelona
1,2,1,103,3,2024-01-02,Teclado,80,Accesorios,240,Ana,Madrid
2,3,5,102,1,2024-01-03,Mouse,25,Accesorios,25,Luis,Sevilla
3,4,1,104,2,2024-01-04,Monitor,300,Electrónica,600,Ana,Madrid
4,5,3,104,4,2024-01-05,Monitor,300,Electrónica,1200,María,Madrid


In [11]:
# Análisis por ciudad
ventas_por_ciudad = analisis_completo.groupby('ciudad')['total'].sum()

print("Ventas totales por ciudad:")
ventas_por_ciudad

Ventas totales por ciudad:


ciudad
Barcelona     235
Madrid       2040
Sevilla       150
Valencia     1200
Name: total, dtype: int64

In [12]:
# Clientes con compras > 1000
clientes_top = analisis_completo.groupby(['id_cliente', 'nombre'])['total'].sum()
clientes_top = clientes_top[clientes_top > 1000]
print(f"\nClientes con compras > 1000:\n{clientes_top}")

KeyError: 'nombre'

In [13]:
clientes_top = analisis_completo.groupby(
    ['id_cliente', 'nombre_y']
)['total'].sum()

clientes_top = clientes_top[clientes_top > 1000]

print("\nClientes con compras > 1000:")
print(clientes_top)


Clientes con compras > 1000:
id_cliente  nombre_y
3           María       1200
4           Juan        1200
Name: total, dtype: int64


In [14]:
ejercicio_dia4_analisis_completo.xlsx

NameError: name 'ejercicio_dia4_analisis_completo' is not defined

In [15]:
# Exportar todas las tablas a un archivo Excel
with pd.ExcelWriter("ejercicio_dia4_analisis_completo.xlsx", engine="openpyxl") as writer:
    
    ventas.to_excel(writer, sheet_name="ventas", index=False)
    clientes.to_excel(writer, sheet_name="clientes", index=False)
    productos.to_excel(writer, sheet_name="productos", index=False)
    
    ventas_recientes.to_excel(writer, sheet_name="ventas_recientes", index=False)
    productos_caros.to_excel(writer, sheet_name="productos_caros", index=False)
    
    ventas_por_producto.reset_index().to_excel(
        writer, sheet_name="ventas_por_producto", index=False
    )
    
    stats_por_cliente_plano.to_excel(
    writer, sheet_name="stats_por_cliente", index=False
)
    
    ventas_productos.to_excel(
        writer, sheet_name="ventas_productos", index=False
    )
    
    analisis_completo.to_excel(
        writer, sheet_name="analisis_completo", index=False
    )
    
    ventas_por_ciudad.reset_index().to_excel(
        writer, sheet_name="ventas_por_ciudad", index=False
    )
    
    clientes_top.reset_index().to_excel(
        writer, sheet_name="clientes_top", index=False
    )

print("Archivo Excel creado correctamente")

NotImplementedError: Writing to Excel with MultiIndex columns and no index ('index'=False) is not yet implemented.

In [16]:
stats_por_cliente_plano = stats_por_cliente.copy()

stats_por_cliente_plano.columns = [
    f"{col[0]}_{col[1]}" if isinstance(col, tuple) else col
    for col in stats_por_cliente_plano.columns
]

stats_por_cliente_plano = stats_por_cliente_plano.reset_index()

In [17]:
with pd.ExcelWriter("ejercicio_dia4_analisis_completo.xlsx", engine="openpyxl") as writer:
    
    ventas.to_excel(writer, sheet_name="ventas", index=False)
    clientes.to_excel(writer, sheet_name="clientes", index=False)
    productos.to_excel(writer, sheet_name="productos", index=False)
    
    ventas_recientes.to_excel(writer, sheet_name="ventas_recientes", index=False)
    productos_caros.to_excel(writer, sheet_name="productos_caros", index=False)
    
    ventas_por_producto.reset_index().to_excel(
        writer, sheet_name="ventas_por_producto", index=False
    )
    
    stats_por_cliente_plano.to_excel(
        writer, sheet_name="stats_por_cliente", index=False
    )
    
    ventas_productos.to_excel(
        writer, sheet_name="ventas_productos", index=False
    )
    
    analisis_completo.to_excel(
        writer, sheet_name="analisis_completo", index=False
    )
    
    ventas_por_ciudad.reset_index().to_excel(
        writer, sheet_name="ventas_por_ciudad", index=False
    )
    
    clientes_top.reset_index().to_excel(
        writer, sheet_name="clientes_top", index=False
    )

print("Archivo Excel creado correctamente")

Archivo Excel creado correctamente
